## 1. Two Sum (Easy)

Given an array of integers `nums` and an integer `target`, return indices of the two numbers such that they add up to `target`.

You may assume that each input would have ***exactly one solution***, and you may not use the same element twice.

You can return the answer in any order


**Example 1:**
```
    Input: nums = [2,7,11,15], target =9
    Output: [0, 1]
    Output: Because nums[0] + nums[1] == 9, we return [0, 1]
```
**Example 2:**
```
    Input: nums = [3,2,4], target = 6
    Output: [1,2]
```
**Example 3:**
```
    Input: nums = [3,3], target = 6
    Output: [0,1]
```
**Constraints:**
```
* 2 <= nums.length <= 103
* -109 <= nums[i] <= 109
* -109 <= target <= 109
*Only one valid answer exists.
```

In [1]:
class Solution:
    def twoSum(self, nums, target):
        map = dict()
        for i in range(len(nums)):
            complement = target - nums[i]
            if complement in map:
                return [map[complement], i]
            map[nums[i]] = i


## 146. LRU Cache (Medium)

Design a data Structure that follows the constraints of a Least Recently Used (LRU) cache.

Implement `LRUCache` class:
* `LRUCache(int capacity)` Initialize the LRU cache with positive size `capacity`
* `int get(int key)` Return the value of the `key` if the key exists, otherwise return -1
* `void put(int key, int value)` Update the value of the `key` if `key` exists. Otherwise, add the `key-value` pair to the cache. If the number of keys exceeds the `capacity` form this operation, **evict** the least recently used key.

**Follow up:**
Could you do `get` and `put` in `O(1)` time complexity?

**Example 1:**

```
Input
["LRUCache", "put", "put", "get", "put", "get", "put", "get", "get", "get"]
[[2], [1, 1], [2, 2], [1], [3, 3], [2], [4, 4], [1], [3], [4]]
Output
[null, null, null, 1, null, -1, null, -1, 3, 4]

Explanation
LRUCache lRUCache = new LRUCache(2);
lRUCache.put(1, 1); // cache is {1=1}
lRUCache.put(2, 2); // cache is {1=1, 2=2}
lRUCache.get(1);    // return 1
lRUCache.put(3, 3); // LRU key was 2, evicts key 2, cache is {1=1, 3=3}
lRUCache.get(2);    // returns -1 (not found)
lRUCache.put(4, 4); // LRU key was 1, evicts key 1, cache is {4=4, 3=3}
lRUCache.get(1);    // return -1 (not found)
lRUCache.get(3);    // return 3
lRUCache.get(4);    // return 4
```
**Constraints:**
* 1 <= capacity <= 3000
* 0 <= key <= 3000
* 0 <= value <= 104
* At most 3 * 104 calls will be made to get and put.

In [2]:
# OrderedDict
from collections import OrderedDict
class LRUCache(OrderedDict):

    def __init__(self, capacity):
        """
        :param capacity: int
        """
        self.capacity = capacity

    def get(self, key):
        """
        :param key: int
        :return: int
        """
        if key not in self:
            return -1
        self.move_to_end(key)
        return self[key]

    def put(self, key, value):
        """
        :param key: int
        :param value: int
        :return: void
        """
        if key in self:
            self.move_to_end(key)
        self[key] = value
        if len(self) > self.capacity:
            self.popitem(last=False)

In [3]:
# Hashmap + DoubleLinkedList
class DLinkedNode():
    def __init__(self):
        self.key = 0
        self.value = 0
        self.prev = None
        self.next = None

class LRUCache():
    def _add_node(self, node):
        """
        Always add the new node right after head.
        :param node:
        :return:
        """
        node.prev = self.head
        node.next = self.head.next

        self.head.next.prev = node
        self.head.next = node

    def _remove_node(self, node):
        """
        Remove an existing node from the linked list
        :param node:
        :return:
        """
        prev = node.prev
        new = node.next

        prev.next = new
        new.prev = prev

    def _move_to_head(self, node):
        """
        Move Certain node in between to the head.
        :param node:
        :return:
        """
        self._remove_node(node)
        self._add_node(node)

    def _pop_tail(self):
        """
        Pop the current tail
        :return:
        """
        res = self.tail.prev
        self._remove_node(res)
        return res

    def __init__(self, capacity: int):
        """
        :param capacity: int
        """
        self.cache = {}
        self.size = 0
        self.capacity = capacity
        self.head, self.tail = DLinkedNode(), DLinkedNode()

        self.head.next = self.tail
        self.tail.prev = self.head

    def get(self, key):
        """
        :param key: int
        :return: int
        """
        node = self.cache.get(key, None)
        if not node:
            return -1
        self._move_to_head(node)
        return node.value

    def put(self, key, value):
        """
        :param key: int
        :param value: int
        :return: None
        """
        node = self.cache.get(key)

        if not node:
            newNode = DLinkedNode()
            newNode.key = key
            newNode.value = value

            self.cache[key] = newNode
            self._add_node(newNode)
            self.size += 1

            if self.size > self.capacity:
                tail = self._pop_tail()
                del self.cache[tail.key]
                self.size -= 1
        else:
            node.value = value
            self._move_to_head(node)


## 42. Trapping Rain Water (Hard)

Given `n` non-negative integers representing an elevation map where the width of each bar is `1`, compute how much water it can trap after raining

**Example 1:**
```
Input: height = [0,1,0,2,1,0,1,3,2,1,2,1]
Output: 6
Explanation: The above elevation map (black section) is represented by array [0,1,0,2,1,0,1,3,2,1,2,1]. In this case, 6 units of rain water (blue section) are being trapped.
```
**Example 2:**
```
Input: height = [4,2,0,3,2,5]
Output: 9
```
**Constraints:**
* n == height.length
* 0 <= n <= 3 * 104
* 0 <= height[i] <= 105

In [4]:
# Dynamic Programming:
"""
1. Find maximum height of bar from the left end upto an index i in the array `left_max`
2. Find maximum height of bar from the right end upto an index i in the array `right_max`
3. Iterate over height array and update ans: Add min(left_max[i], right_max[i]) - height[i] to ans
Time Complexity: O(n), Space Complexity: O(n) extra space
"""

class Solution:
    def trap(self, height):
        """
        :param height: List[int]
        :return: int
        """
        if not height:
            return 0
        ans = 0
        size = len(height)
        left_max, right_max = [0] * size, [0] * size

        left_max[0] = height[0]
        for i in range(1, size):
            left_max[i] = max(height[i], left_max[i-1])

        right_max[size-1] = height[size-1]
        for i in reversed(range(size-1)):
            right_max[i] = max(height[i], right_max[i+1])

        for i in range(1, size-1):
            ans +=  min(left_max[i], right_max[i]) - height[i]
        return ans

In [1]:
# Using Stacks
"""
We keep a stack and iterate over the array.
We add the index of the bar to the stack if bar <= top of stack, which mean the current bar is bounded by the previous bar in the stack
If we found a current bar > the top,  we are sure that the bar at the top of the stack is bounded by the current bar and previous bar in the stack.
Hence, we can pop it and add resulting trapped water to `ans`
Time Complexity: O(n), Space Complexity: O(n)
"""
class Solution:
    def trap(self, height):
        """
        :param height: List[int]
        :return: int
        """
        ans, current = 0, 0
        st = []
        while (current < len(height)):
            while (st and height[current]>height[st[-1]]):
                top = st.pop()
                if len(st) == 0:
                    break
                distance = current - st[-1] - 1
                bounded_height = min(height[current], height[st[-1]]) - height[top]
                ans += distance * bounded_height
            st.append(current)
            current+=1
        return ans

In [6]:
# Using 2 Pointers
"""
If there is a larger bar at one end (say right), we are assured that the water trapped would be dependant on height of bar
in current direction (from left ot right).
As soon as we find the bar at other end (right) is smaller, we start iterating in opposite direction (from right to left).
We must maintain left_max and right_max during the iteration, but now we can do it in one iteration using 2 pointers, switching between two
Time complexity: O(n), Space complexity: O(1)
"""
class Solution:
    def trap(self, height):
        """
        :param height: List[int]
        :return: int
        """
        left, right = 0, len(height)-1
        ans = 0
        left_max, right_max = 0, 0
        while left < right:
            if height[left] < height[right]:
                if height[left] > left_max:
                    left_max = height[left]
                else:
                    ans += left_max - height[left]
                left += 1
            else:
                if height[right] > right_max:
                    right_max = height[right]
                else:
                    ans += right_max - height[right]
                right -= 1
        return ans

## 200. Number of Islands (Medium)
Given an `mxn` 2D binary grid `grid` which represents a map of `'1'`s (land) and `'0'`s (water), return the number of islands.

An **island** is surrounded by water and is formed by connecting adjacent lands horizontally or vertically. You may assume all four edges of the grid are all surrounded by water

**Example 1:**
```
Input: grid = [
  ["1","1","1","1","0"],
  ["1","1","0","1","0"],
  ["1","1","0","0","0"],
  ["0","0","0","0","0"]
]
Output: 1
```
**Example 2:**
```
Input: grid = [
  ["1","1","0","0","0"],
  ["1","1","0","0","0"],
  ["0","0","1","0","0"],
  ["0","0","0","1","1"]
]
Output: 3
```

**Constraints:**
* m == grid.length
* n == grid[i].length
* 1 <= m, n <= 300
* grid[i][j] is '0' or '1'.

In [4]:
# DFS (Depth First Search): using recursive function
# Time Complexity: O(MxN), Space Complexity: O(MxN)
class Solution:
    def dfs(self, grid, r, c):
        nr = len(grid)
        nc = len(grid[0])
        if r < 0 or c < 0 or r >= nr or c >= nc or grid[r][c] == '0':
            return
        grid[r][c]='0'
        self.dfs(grid, r-1, c)
        self.dfs(grid, r+1, c)
        self.dfs(grid, r, c-1)
        self.dfs(grid, r, c+1)
    def numIslands(self, grid):
        """
        :param grid: List[List[str]
        :return: int
        """
        if len(grid) == 0:
            return 0
        nr = len(grid)
        nc = len(grid[0])
        num_islands = 0
        for r in range(nr):
            for c in range(nc):
                if grid[r][c] == '1':
                    num_islands += 1
                    self.dfs(grid, r, c)
        return num_islands

In [5]:
# BFS (Breadth First Search): Using Queue
# Time Complexity: O(MxN), Space Complexity: O(min(M,N))

class Solution:
    def numIslands(self, grid):
        if len(grid) == 0:
            return 0
        nr = len(grid)
        nc = len(grid[0])
        num_islands=0
        for r in range(nr):
            for c in range(nc):
                if grid[r][c]=='1':
                    num_islands +=1
                    neighbors = []
                    # neighbors.append((r,c))
                    neighbors.append(r*nc + c)
                    while neighbors:
                        # rc = neighbors.pop(0)
                        # row, col = rc[0], rc[1]
                        id = neighbors.pop(0)
                        row, col = id // nc, id % nc
                        if row - 1 >= 0 and grid[row-1][col] == '1':
                            neighbors.append((row-1)*nc + col)
                            grid[row-1][col] = '0'
                        if row + 1 < nr and grid[row+1][col] == '1':
                            neighbors.append((row+1)*nc + col)
                            grid[row+1][col] = '0'
                        if col - 1 >= 0 and grid[row][col-1] == '1':
                            neighbors.append(row*nc + col-1)
                            grid[row][col-1] = '0'
                        if col + 1 < nc and grid[row][col+1] == '1':
                            neighbors.append(row*nc + col+1)
                            grid[row][col+1] = '0'
        return num_islands

## 56. Merge Intervals (Medium)
Given an array of `intervals` where ```intervals[i] = [start_i, end_i]```, merge all overlapping intervals, and return an array of the non-overlapping inverals that cover all the intervals in the input.

**Example 1:**
```
Input: intervals = [[1,3],[2,6],[8,10],[15,18]]
Output: [[1,6],[8,10],[15,18]]
Explanation: Since intervals [1,3] and [2,6] overlaps, merge them into [1,6].
```
**Example 2:**
```
Input: intervals = [[1,4],[4,5]]
Output: [[1,5]]
Explanation: Intervals [1,4] and [4,5] are considered overlapping.
```
**Constraints:**
* 1 <= intervals.length <= 10^4
* intervals[i].length == 2
* 0 <= start_i <= end_i <= 10^4

In [ ]:
# Sorting
# Time Complexity: O(NlogN), Space Complexity: O(N)(O(logN))
class Solution:
    def merge (self, intervals):
        intervals.sort(key=lambda x: x[0])
        merged = []
        for interval in intervals:
            # if the list of merged intervals is empty or if the current
            # interval does not overlap with previous, simply append it.
            if not merged or merged[-1][1] < interval[0]:
                merged.append(interval)
            else:
            # otherwise, there is overlap, so we merge the current and
            # previous intervals
                merged[-1][1] = max(merged[-1][1], interval[1])
        return merged

## 1041. Robot Bounded in Circle (Medium)
On an infinite plane, a robot initially stands at `(0,0)` and faces north. The robot can receive one of three instructions:
* `"G"`: go straight 1 unit;
* `"L"`: turn 90 degrees to the left;
* `"R"`: turn 90 degrees to the right;

THe robot performs the `instructions` given in order, and repeats them forever.

Return `true` if and only if there exists a circle in the plane such that robot never leaves the circle.

**Example 1:**
```
Input: instructions = "GGLLGG"
Output: true
Explanation: The robot moves from (0,0) to (0,2), turns 180 degrees, and then returns to (0,0).
When repeating these instructions, the robot remains in the circle of radius 2 centered at the origin.
```
**Example 2: **
```
Input: instructions = "GG"
Output: false
Explanation: The robot moves north indefinitely.
```

In [7]:
# One Pass
# After at most 4 cycles, the limit cycle trajectory returns to the initial point `x=0`, `y=0`.
# That is related to the fact that 4 direction (north, east, south, west) define the repeated
# cycles' plane symmetry
# We do not need to run 4 cycles to identify the limit cycle trajectory. One cycle is enough
#   1. If the robot returns to the initial point after one cycle, that's the limit cycle trajectory.
#   2. If the robot doesn't face north at the end of the first cycle, that's the limit cycle trajectory. Once again,
#       that's the consequence of the plane symmetry for the repeated cycles.
# Time Complexity: O(N) - N is number of instructions to parse
# Space complexity: O(1) - the array contains only 4 elements
class Solution:
    def isRobotBounded(self, instructions):
        """
        :param instructions: str
        :return: bool
        """
        # north = 0, east = 1, south = 2, west = 3
        directions = [[0, 1], [1, 0], [0, -1], [-1, 0]]
        # Initial position is in the center
        x = y = 0
        # facing north
        idx = 0
        for i in instructions:
            if i == "L":
                idx = (idx + 3) % 4
            elif i == "R":
                idx = (idx + 1) % 4
            else:
                x += directions[idx][0]
                y += directions[idx][1]
        # after one cycle:
        # robot returns into initial position
        # or robot doesn't face north
        return (x == 0 and y == 0) or idx != 0

## 68. Text Justification (Hard)

Given an array of words and a width `maxWidth`, format the text such that each line has exactly `maxWidth` characters and is fully (left and right) justified.

You should pack your words in a greedy approach; that is, pack as many words as you can in each line. Pad extra spaces `' '` when necessary so that each line has exactly `maxWidth` characters.

Extra spaces between words should be distributed as evenly as possible. If the number of spaces on a line do not divide evenly between words, the empty slots on the left will be assigned more spaces than the slots on the right.

For the last line of text, it should be left justified and no **extra** space is inserted between words.

**Note:**
* A word is defined as a character sequence consisting of non-space characters only.
* Each word's length is guaranteed to be greater than 0 and not exceed `maxWidth`.
* The input array `words` contains at least one word.

**Example 1:**
```
Input: words = ["This", "is", "an", "example", "of", "text", "justification."], maxWidth = 16
Output:
[
   "This    is    an",
   "example  of text",
   "justification.  "
]
```
**Example 2:**
```
Input: words = ["What","must","be","acknowledgment","shall","be"], maxWidth = 16
Output:
[
  "What   must   be",
  "acknowledgment  ",
  "shall be        "
]
Explanation: Note that the last line is "shall be    " instead of "shall     be", because the last line must be left-justified instead of fully-justified.
Note that the second line is also left-justified becase it contains only one word.
```
**Example 3:**
```
Input: words = ["Science","is","what","we","understand","well","enough","to","explain","to","a","computer.","Art","is","everything","else","we","do"], maxWidth = 20
Output:
[
  "Science  is  what we",
  "understand      well",
  "enough to explain to",
  "a  computer.  Art is",
  "everything  else  we",
  "do                  "
]
```
**Constraints:**
* 1 <= words.length <= 300
* 1 <= words[i].length <= 20
* words[i] consists of only English letters and symbols.
* 1 <= maxWidth <= 100
* words[i].length <= maxWidth

In [ ]:
class Solution:
    def fullJustify(self, words, maxWidth):
        res, cur, num_of_letters = [], [], 0
        for w in words:
            if num_of_letters + len(w) + len(cur) > maxWidth:
                #  Once you determine that there are only k words that can fit on a given line,
                #  you know what the total length of those words is num_of_letters.
                #  Then the rest are spaces, and there are (maxWidth - num_of_letters) of spaces.
                #  The "or 1" part is for dealing with the edge case len(cur) == 1.
                for i in range(maxWidth - num_of_letters):
                    cur[i%(len(cur)-1 or 1)] += ' '
                res.append(''.join(cur))
                cur, num_of_letters = [], 0
            cur += [w]
            num_of_letters += len(w)
        return res + [' '.join(cur).ljust(maxWidth)]


## 91. Decode Ways (Medium)

A message containing letters from `A-Z`can be **encoded** into numbers using the following mapping:
```
'A' -> "1"
'B' -> "2"
...
'Z' -> "26"
```
To **decode** an encoded message, all the digits must be grouped then mapped back into letters using the reverse of the mapping above (there may be multiple ways). For example, `"11106"` can be mapped into:
* `"AAJF"` with the grouping `(1 1 10 6)`
* `"KJF"` with the grouping `(11 0 6)`
Note that the grouping `(1 11 06)` is invalid because `"06"` cannot be mapped into `'F'` since `"6"` is different from `"06"`.

Given a string `s` containing only digits, return the ***number** of ways to **decode** it*.

The answer is guaranteed to fit in a **32-bit** integer.

**Example 1:**
```
Input: s = "12"
Output: 2
Explanation: "12" could be decoded as "AB" (1 2) or "L" (12).
```
**Example 2:**
```
Input: s = "226"
Output: 3
Explanation: "226" could be decoded as "BZ" (2 26), "VF" (22 6), or "BBF" (2 2 6).
```
**Example 3:**
```
Input: s = "0"
Output: 0
Explanation: There is no character that is mapped to a number starting with 0.
The only valid mappings with 0 are 'J' -> "10" and 'T' -> "20", neither of which start with 0.
Hence, there are no valid ways to decode this since all digits need to be mapped.
```
**Example 4:**
```
Input: s = "06"
Output: 0
Explanation: "06" cannot be mapped to "F" because of the leading zero ("6" is different from "06").
```
**Constraints:**
* 1 <= s.length <= 100
* `s` contains only digits and may contain leading zero(s).

In [ ]:
# Recursive Approach with Memorization
# Time Complexity: O(N) - Memorization with LRU cache helps in pruning the recursion tree and hence decoing for an index only once.
# Space Complexity: O(N) - The dictionary used for memorization would take the space equal to the length of the string
from functools import lru_cache
class Solution:
    @lru_cache(maxsize=None)
    def recursiveWithMemo(self, index, s) -> int:
        # If you reach the end of the string
        # Return 1 for success.
        if index == len(s):
            return 1
        # If the string starts with zero, it can't be decoded
        if s[index] == '0':
            return 0

        if index == len(s)-1:
            return 1

        answer = self.recursiveWithMemo(index + 1, s)
        if int(s[index:index + 2]) <= 26:
            answer += self.recursiveWithMemo(index+2,s)
        return answer

    def numDecodings(self, s: str) -> int:
        return self.recursiveWithMemo(0, s)

In [ ]:
# Interative Approach (dynamic programming array)
# Time Complexity: O(N)
# Space Complexity: O(N). The length of the DP array.
class Solution:
    def numDecodings(self, s: str) -> int:
        # Array to store the subproblem results
        dp = [0 for _ in range(len(s) + 1)]
        dp[0] = 1
        # Ways to decode a string of size 1 is 1 unless the string is '0'.
        # '0' doesn't have a single digit decode
        dp[1] = 0 if s[0] == '0' else 1

        for i in range(2, len(dp)):
            # Check if successful single digit decodes is possible.
            if s[i-1] != '0':
                dp[i] = dp[i-1]
            # Cehck if successful two digit decode is possible
            two_digit = int(s[i-2: i])
            if two_digit >= 10 and two_digit <= 26:
                dp[i] += dp[i-2]
        return dp[len(s)]

In [ ]:
# Iterative, Constant Space
# We only back only two steps in DP array in the previous Approach.
# Time Complexity: O(N)
# Space Complexity O(1), simply using two variables
class Solution:
    def numDecodings(self, s:str) -> int:
        if s[0] == "0":
            return 0

        two_back = 1
        one_back = 1
        for i in range(1, len(s)):
            current = 0
            if s[i] != "0":
                current = one_back
            two_digit = int(s[i-1: i+1])
            if two_digit >= 10 and two_digit <= 26:
                current += two_back
            two_back = one_back
            one_back = current
        return one_back

